# Foreword

For decades, a large part of the increase of the performance of CPU was driven by the rapid inflation of the number of transistors (roughly doubling the number of transistors per unit area every couple of years) going in hand in hand with a constant boost of the clock frequency. Unfortunately over the past decade progress has been modest. The miniaturization of transitors has become increasingly more difficult and heat dissipation is now a major issue preventing further increase in clock frequency. To keep improving the performance, CPU makers have switched to an alternative strategy, putting several processing cores in a single CPU die, yielding in principle speed improvements proportional to the number of cores. However writing a software that efficiently uses several cores can be (much) more difficult than simply using just one core.

Because of some fundamental elements in its architecture (see [GIL](https://en.wikipedia.org/wiki/Global_interpreter_lock)), the main Python implementation, CPython, makes it very difficult to write efficient parallel programs in Python. Over the years, various methods have been put forward to overcome this issue. I present here one such approach: process-based parallelism. This is a high-level parallelism. Rather than parallelizing individual mathematical operations, we do so at the highest level possible. This is ideal for running many independent tasks such as computing models over a grid (each process computes entirely one model based on different input parameters, with several processes doing so in parallel), fitting these grids of models to different objects (each process does all the fitting for one object, with several processes doing so in parallel), or generating series of plots for instance (each process entirely generates one plot, with several processes doing so in parallel).

Note: the examples presented below are generally not optimal as for the sake of demonstration they do only very simple things and we are most likely dominated by overheads. For the best gain possible, the parallelization should be done on the longest running tasks possible.

---

# Initial setup and a warning

We will implement parallel computation using python's multiprocessing module. When starting a parallel computation it forks (Linux/OS X) or spawns (Windows) a new process. However from experience when Numpy is linked with certain optimized numerical libraries, using fork triggers a deadlock on OS X (in theory on Linux too but I have never run into the problem). To avoid that we will settle on using spawn, which creates entirely new processes independent from the parent process (fork basically just does a minimal cloning, keeping a lot of shared elements). This will require Python 3.4+. If you have an earlier version, it may or may not work. The downside of using the spawn start method is that it is much slower than fork to generate the child processes. So one needs to make sure that it is only used for long running tasks.

Another important point is that the multiprocessing module does not get along well with interactive python sessions when the functions to be run in parallel are defined within the interactive session. To work around this issue, we import these functions from the pyastro16 module. Feel free to peruse and modify this module at your leisure.

In [ ]:
import multiprocessing as mp
import numpy as np
import pyastro16 # Make sure it is in the same directory as this notebook

mp.set_start_method('spawn')

print("This CPU has {} cores.".format(mp.cpu_count()))

---

# A few simple parallel programs

Let's start with a simple practical example, initializing an array with $\sin\left(\theta\right)$ for $\theta\in\left[0; 2\pi\right]$.

In [ ]:
theta = np.linspace(0., 2. * np.pi, 100)

with mp.Pool() as pool:
    results = pool.map(np.sin, theta)

The pool variable contains the pool of processes that python can tap into. Pool() called without argument will create as many processes as there are logical cores in your computer. Then pool.map simply applies the input values (second argument) to the function (first argument). To do so it chops the input values into blocks that are fed separately to the different processes. The results are stored in a list. Note that the call is blocking until all the computations are done (if you are looking for something asynchronous, [multiprocessing also provides the relevant functions](https://docs.python.org/3/library/multiprocessing.html#module-multiprocessing.pool)). As we can see the results look correct.

In [ ]:
%matplotlib notebook
from matplotlib import pyplot as plt

plt.figure()
plt.plot(theta, results)

Now let's try something slightly more complicated. In the previous example we only used one input parameter. However in practice one may need to carry out a computation depending on several parameters. Unfortunately Pool.map() only takes one argument. But this can easily be solved using Pool.starmap() rather.

In [ ]:
theta = np.linspace(0., 2. * np.pi, 100)
scale = np.exp(-theta)

with mp.Pool() as pool:
    results = pool.starmap(pyastro16.scaled_sin, zip(theta, scale))

plt.figure()
plt.plot(theta, results)

---
# Sharing data between processes

So far we have remained in a simple case: the parallel functions just take one or a few scalar arguments and the results are provided in the form of a list return by Pool.map() or Pool.starmap(). However it can be the case that these functions need a lot more information than this (for instance large Numpy arrays containing spectra, light curves, etc.) and/or that they return several quantities that would be better stored directly into a Numpy array (lists are not only slow compare to Numpy arrays for numerical data, they are also quite inefficient in terms of RAM usage, which can be a problem when you output GB worth of data).

A naïve solution would be to pass these arrays as arguments. However as we use the "spawn" method each child process would immediately make a full separate copy of it for its exclusive usage. Not only is this slow, it could also rapidly engulf a significant fraction of the RAM if the arrays are not small and in any case it means that we cannot use such arrays to store the results.

The multiprocessing.sharedctypes module actually provides us with objects that can be shared between children processes: Value and Array, which contain respectively a scalar and (surprise!) an array. They come with a lock to read or write the data. This is useful if you read/write an element with one process that can be read/written by another process. If you read an array that is never modified or write to an array where you are certain that no two processes will ever write at the same location, then you can rather use RawValue and RawArray. These functions return a c_type object, which is basically a pointer to an appropriately sized block of RAM. There are two obvious problems that pop up here. First, we want to deal with a Numpy array and all its niceties, not a cold, raw block of RAM. The second problem is that (Raw)Array have no notion of the shape of the array. We will deal with these issues in time. But let's start with a very simple example first as there are a few delicate points to get this working properly.


In [ ]:
from multiprocessing.sharedctypes import RawValue, Value

counter = Value('i', 0) # Simple shared integer variable starting at 0 (with lock)
with mp.Pool(initializer=pyastro16.counter_init, initargs=(counter,)) as pool:
    pool.map(pyastro16.counter, range(100000))
print("The final value of the counter with a lock is {}.".format(counter.value))

counter = RawValue('i', 0) # Simple shared integer variable starting at 0 (without lock)
with mp.Pool(initializer=pyastro16.counter_init, initargs=(counter,)) as pool:
    pool.map(pyastro16.counter_nolock, range(100000))
print("The final value of the counter without a lock is {}.".format(counter.value))

Here we have created a simple shared counter. Each time a child process is called it locks the object to make sure no other child process can modify the counter, it increments the counter, and finally it releases the lock. The second version increments without invoking a lock. We see that the final count is not correct in such case. This is because several child processes incremented the counter at the same time from the same initial value. Beyond these findings, there are also two important and related points to notice: 1. the counter is not passed as an argument, 2. mp.Pool() was called with two arguments. Because counter is a variable that will be shared between processes, it cannot really be considered as an argument. Rather it should be seen as a sort of global variable. This is where the initializer and initargs come into play. If counter is a global variable, the child processes must be made aware of its existence. The initializer argument takes a function. This function is called once when each child process is created. It takes the arguments from initargs. The trick then is to use the initializer to declare the initarg variables as global variables. Then they will be accessible to read/write within each child process while actually existing in only one location in RAM so they are effectively shared across child processes.

Now that we know how to share variables between all the child processes, we can go back to the question of dealing with (multidimensional) Numpy arrays rather than multiprocessing.sharedctypes arrays. So we need to find a way to tell the child processes to convert the (Raw)Arrays into Numpy arrays of any dimension without losing their shared-memory properties. For that we do two things. First, instead to passing the (Raw)Array to the initializer we pass a tuple containing the (Raw)Array and its dimension. Then in the initializer we read the (Raw)Array as a Numpy array and we reshape this array into the right dimension. In effect the Numpy array points to the same memory block as the (Raw)Array. Here is a simple example where we use such a strategy to create random numbers.

In [ ]:
import ctypes
from multiprocessing.sharedctypes import RawArray, Value

npar = 100
noutput = 100000

output = (RawArray(ctypes.c_double, npar * noutput), (npar, noutput))
counter = Value('i', 0)

with mp.Pool(initializer=pyastro16.random_init, initargs=(output, counter)) as pool:
    pool.starmap(pyastro16.random, enumerate(np.linspace(2., 3., npar)))

Note that here we only use shared arrays to store the results of the computation. However we could very well pass arrays that are not meant to be modified but that already contain data. Think of passing arrays containing models and observations to a fitting procedure for instance.

---

# Why the speedup is not always proportional to the number of cores

Ideally, the speedup brought by multiprocessing should be proportional to the number of cores. However this is not always reached. For instance as we can see, if we compare the gain running the previous example in parallel or not, the gain is not as much as one could expect.

In [ ]:
from itertools import starmap

def compute_serial():
    pyastro16.random_init(output, counter)
    list(starmap(pyastro16.random, enumerate(np.linspace(2., 3., npar))))

In [ ]:
%timeit compute_serial()

In [ ]:
def compute_ncores(ncores):
    with mp.Pool(processes=ncores, initializer=pyastro16.random_init, initargs=(output, counter)) as pool:
        pool.starmap(pyastro16.random, enumerate(np.linspace(2., 3., npar)))

In [ ]:
# Sorry for the ugly repetition, timeit does not like it otherwise
%timeit compute_ncores(1)
%timeit compute_ncores(2)
%timeit compute_ncores(3)
%timeit compute_ncores(4)
%timeit compute_ncores(5)
%timeit compute_ncores(6)
%timeit compute_ncores(7)
%timeit compute_ncores(8)

We should obtain the best result when the number of processes is equal to the number of logical cores, we will see later why. However as we can see, the gain in execution time is not exactly proportional to the number of physical coresused. There are several possible reasons for that. It can be important to understand them as they can guide towards a better strategy.

## Large startup overheads

Using multiprocessing induces large startup overheard simply because starting child processes with the spawn method can take of the order of hundreds of ms. This means that the total run time should at least be a few times this value for the parallelization to be worthwhile. This also suggests that the parallelization should be done at the highest level possible. For instance when fitting models to data from different objects, it is more efficient that each child process handles the whole fitting process for one object rather than fitting objects in series and parallelizing only specific parts of the fitting process (for example the computation of the $\chi^2$, etc.). If you cannot parallelize at a high level, multiprocessing is probably not the method you want to use to parallelize your computation and you may be better off with OpenMP/MPI for instance.

## Saturating the memory bandwidth

When doing fast operations on large arrays, each processor may need to transfer large amounts of data to and from the RAM. Even if modern RAM is not very fast compared to the CPU, this is generally not a major problem when running on only one core. However when running on multiple cores, they compete to access the RAM and they can end up spending a non negligible fraction of their time waiting for the RAM to send the data over. Make sure you access the memory efficiently, reading contiguous blocks. Naturally having a computer with a faster memory such as DDR4 rather than DDR3 won't hurt.

## Physical core ≠ logical core

Modern top-of-the-line processors from Intel integrate the so-called [Hyperthreading](https://en.wikipedia.org/wiki/Hyper-threading) technology. For each physical core, this creates two virtual logical cores. This allows the operating system to optimize the scheduling of processing, for instance by scheduling out a process that is waiting data from the RAM to schedule in another process whose requested data has arrived in the meantime. The actual gain is very dependent on the workload but in my experience it is a good rule of thumb to parallelize with as many processes as there are logical cores. Going further is detrimental and will likely slow down everything as processes have to be scheduled in and out more often than necessary.

## Frequency scaling

Modern processors run at a variable frequency depending on the load. This is done to limit thermal dissipation. This means that when running a single core, it is possible to run at a higher base frequency than when running with multiple cores. Just from this effect, running on n cores rather than one will not yield a speedup of a factor n.
